In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import importlib
import pandas as pd
import cv2
import numpy as np

In [2]:
class Arguments():
    def __init__(self):
        self.images = 3012
        self.epochs = 100
        self.clients = 1
        self.local_batches = 20
        self.lr = 0.01
        self.iid = 'iid'
        self.dropout1 = 0.25
        self.dropout2 = 0.5
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 100
        self.use_cuda = True
        self.save_model = False
        self.save_model_interval = 200
        self.clip = 1

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [3]:
class CovidDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        self.data_info = pd.read_csv(csv_path, header=None)
        self.transform = transform
                
    def __len__(self):
        return len(self.data_info)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.data_info.iloc[idx, 0]
        image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        label = self.data_info.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        img = cv2.resize(image, (self.output_size, self.output_size))

        return {'image': img, 'label': label}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        
        tensor_img = torch.from_numpy(image)
        tensor_img = tensor_img.unsqueeze(dim=0)
        tensor_img = tensor_img.type('torch.FloatTensor')
        tensor_lb = torch.from_numpy(label)

        return {'image': tensor_img,
                'label': tensor_lb}

In [4]:
global_train_dataset = CovidDataset('./train.csv', transform=transforms.Compose([Rescale(32), ToTensor()]))
global_train_loader = DataLoader(global_train_dataset, batch_size=args.local_batches, shuffle=True, drop_last=True)

In [5]:
global_test_dataset = CovidDataset('./test.csv', transform=transforms.Compose([Rescale(32), ToTensor()]))
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True, drop_last=True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                               out_channels = 32,
                               kernel_size = 3,
                               stride = 1)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1)
        self.fc1 = nn.Linear(14*14*64, 128)
        self.fc1_bn = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
#         x = self.conv1_bn(x)
        x = F.relu(self.conv2(x))
        
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, p=args.dropout1)
        x = x.view(-1, 14*14*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=args.dropout2)
#         x = self.fc1_bn(x)
        x = self.fc2(x)
        return F.softmax(x)

In [7]:
def train(args, device, model, optimizer, criterion, train_loader):
    model.train()
    
    for epoch in range(args.epochs):
        for batch_idx, batch in enumerate(train_loader):
            data = batch['image']
            target = batch['label']
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.squeeze(1))
            loss.backward()
    
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            optimizer.step()
            
            if batch_idx % args.log_interval == 0 or batch_idx==len(train_loader)-1:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * args.local_batches, 
                    len(train_loader) * args.local_batches, 
                    100. * (batch_idx+1) / len(train_loader), 
                    loss.item()/args.log_interval))
                
        acc = test(args, model, device, global_test_loader)

In [8]:
def test(args, model, device, test_loader):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for d in test_loader:
            data = d['image']
            target = d['label']
            data, target = data.to(device), target.to(device)
            if(str(device)=='cuda'):
                model.cuda()
            output = model(data.float())
#             test_loss += F.nll_loss(output, target.squeeze(1), reduction='sum').item() # sum up batch loss
            loss_fn = nn.CrossEntropyLoss(reduction='sum')
            test_loss += loss_fn(output, target.squeeze(1)).item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for  model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [9]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum = 0.8)
criterion = nn.CrossEntropyLoss(reduction='mean')
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc1_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)


In [10]:
print(len(global_train_loader))

150


In [ ]:
train(args, device, model, optimizer, criterion, global_train_loader)

/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/3000 (1%)]	Loss: 0.012480
Train Epoch: 0 [2000/3000 (67%)]	Loss: 0.011485
Train Epoch: 0 [2980/3000 (100%)]	Loss: 0.013014

Test set: Average loss for  model: 1.2021, Accuracy: 242/753 (32%)

Train Epoch: 1 [0/3000 (1%)]	Loss: 0.012014
Train Epoch: 1 [2000/3000 (67%)]	Loss: 0.010514
Train Epoch: 1 [2980/3000 (100%)]	Loss: 0.010484

Test set: Average loss for  model: 1.1819, Accuracy: 258/753 (34%)

Train Epoch: 2 [0/3000 (1%)]	Loss: 0.011014
Train Epoch: 2 [2000/3000 (67%)]	Loss: 0.012514
Train Epoch: 2 [2980/3000 (100%)]	Loss: 0.011514

Test set: Average loss for  model: 1.1991, Accuracy: 245/753 (33%)

Train Epoch: 3 [0/3000 (1%)]	Loss: 0.011514
Train Epoch: 3 [2000/3000 (67%)]	Loss: 0.012014
Train Epoch: 3 [2980/3000 (100%)]	Loss: 0.011506

Test set: Average loss for  model: 1.1900, Accuracy: 252/753 (33%)

Train Epoch: 4 [0/3000 (1%)]	Loss: 0.011014
Train Epoch: 4 [2000/3000 (67%)]	Loss: 0.011514
Train Epoch: 4 [2980/3000 (100%)]	Loss: 0.013014

Test set: Average 


Test set: Average loss for  model: 0.6047, Accuracy: 691/753 (92%)

Train Epoch: 39 [0/3000 (1%)]	Loss: 0.005515
Train Epoch: 39 [2000/3000 (67%)]	Loss: 0.005515
Train Epoch: 39 [2980/3000 (100%)]	Loss: 0.005982

Test set: Average loss for  model: 0.6029, Accuracy: 691/753 (92%)

Train Epoch: 40 [0/3000 (1%)]	Loss: 0.006035
Train Epoch: 40 [2000/3000 (67%)]	Loss: 0.005942
Train Epoch: 40 [2980/3000 (100%)]	Loss: 0.006228

Test set: Average loss for  model: 0.5886, Accuracy: 706/753 (94%)

Train Epoch: 41 [0/3000 (1%)]	Loss: 0.006032
Train Epoch: 41 [2000/3000 (67%)]	Loss: 0.005820
Train Epoch: 41 [2980/3000 (100%)]	Loss: 0.006512

Test set: Average loss for  model: 0.5965, Accuracy: 697/753 (93%)

Train Epoch: 42 [0/3000 (1%)]	Loss: 0.005515
Train Epoch: 42 [2000/3000 (67%)]	Loss: 0.006118
Train Epoch: 42 [2980/3000 (100%)]	Loss: 0.005514

Test set: Average loss for  model: 0.5950, Accuracy: 699/753 (93%)

Train Epoch: 43 [0/3000 (1%)]	Loss: 0.006110
Train Epoch: 43 [2000/3000 (67%)]	